In [2]:
from mec_env_var import *
from helper import *
import tensorflow as tf
import ipdb as pdb
import time

for k in range(10):
    
    tf.reset_default_graph()
    
    print('---------' + str(k) + '------------')
    
    MAX_EPISODE = 100
    MAX_EPISODE_LEN = 10000

    NUM_T = 1
    NUM_R = 4
    SIGMA2 = 1e-9

    t_factor = 0.5
    noise_sigma = 0.12

    config = {'state_dim':3, 'action_dim':2};
    train_config = {'minibatch_size':64, 'actor_lr':0.0001, 'tau':0.001, 
                    'critic_lr':0.001, 'gamma':0.99, 'buffer_size':250000, 
                    'random_seed':int(time.clock()*1000%1000), 'noise_sigma':noise_sigma, 'sigma2':SIGMA2}
    
#     rate = 3.0
    res_path = 't_M_05_nB/'
    model_path = 'model_M_05_nB/my_train_model_'+str(k)+'-2000'
    meta_path = model_path+'.meta'    
    user_config = [{'id':'1', 'model':'AR', 'num_r':NUM_R, 'rate':1.0, 'dis':100, 'action_bound':2, 
                    'data_buf_size':100, 't_factor':t_factor, 'penalty':1000},
                   {'id':'2', 'model':'AR', 'num_r':NUM_R, 'rate':2.0, 'dis':100, 'action_bound':2, 
                    'data_buf_size':100, 't_factor':t_factor, 'penalty':1000},
                   {'id':'3', 'model':'AR', 'num_r':NUM_R, 'rate':3.0, 'dis':100, 'action_bound':2, 
                    'data_buf_size':100, 't_factor':t_factor, 'penalty':1000}]
    
    print(user_config)
    # 0. initialize the session object
    sess = tf.Session() 

    # gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.2)
    # sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options)) 

    # 1. include all user in the system according to the user_config
    user_list = [];
    for info in user_config:
        info.update(config)
        info['model_path'] = model_path
        info['meta_path'] = info['model_path']+'.meta'
        user_list.append(MecTermLD(sess, info, train_config))
        print('Initialization OK!----> user ' + info['id'])

    # 2. create the simulation env
    env = MecSvrEnv(user_list, NUM_R, SIGMA2, MAX_EPISODE_LEN)

#     sess.run(tf.global_variables_initializer())

#     env.init_target_network()

    # #Create a saver object which will save all the variables
    # saver = tf.train.Saver() 

    res_r = []
    res_p = []
    res_b = []
    res_o = []
    res_d = []
    # 3. start to explore for each episode
    for i in range(MAX_EPISODE):

        cur_init_ds_ep = env.reset(isTrain=False)
        
        cur_r_ep = np.zeros(len(user_list))
        cur_p_ep = np.zeros(len(user_list))
        cur_op_ep = np.zeros(len(user_list))
        cur_ts_ep = np.zeros(len(user_list))
        cur_ps_ep = np.zeros(len(user_list))
        cur_rs_ep = np.zeros(len(user_list))
        cur_ds_ep = np.zeros(len(user_list))
        cur_ch_ep = np.zeros(len(user_list))
        cur_of_ep = np.zeros(len(user_list))

        for j in range(MAX_EPISODE_LEN):

            # first try to transmit from current state
            [cur_r, done, cur_p, cur_op, temp, cur_ts, cur_ps, cur_rs, cur_ds, cur_ch, cur_of] = env.step_transmit()

            cur_r_ep += cur_r
            cur_p_ep += cur_p
            cur_op_ep += cur_op
            cur_ts_ep += cur_ts
            cur_ps_ep += cur_ps
            cur_rs_ep += cur_rs
            cur_ds_ep += cur_ds
            cur_ch_ep += cur_ch
            cur_of_ep += cur_of

            if done:
                res_r.append(cur_r_ep/MAX_EPISODE_LEN)
                res_p.append(cur_p_ep/MAX_EPISODE_LEN)
                res_b.append(cur_ds_ep/MAX_EPISODE_LEN)
                res_o.append(cur_of_ep/MAX_EPISODE_LEN)
                res_d.append(cur_ds)
                print('%d:r:%s,p:%s,op:%s,tr:%s,pr:%s,rev:%s,dbuf:%s,ch:%s,ibuf:%s,rbuf:%s' % (i, cur_r_ep/MAX_EPISODE_LEN, cur_p_ep/MAX_EPISODE_LEN, cur_op_ep/MAX_EPISODE_LEN, cur_ts_ep/MAX_EPISODE_LEN, cur_ps_ep/MAX_EPISODE_LEN, cur_rs_ep/MAX_EPISODE_LEN, cur_ds_ep/MAX_EPISODE_LEN, cur_ch_ep/MAX_EPISODE_LEN, cur_init_ds_ep, cur_ds))

    name = res_path+'test_1000_' + time.strftime("%b_%d_%Y_%H_%M_%S", time.localtime(time.time()))
    np.savez(name, res_r, res_p, res_b, res_o, res_d)
    
    sess.close()


---------0------------
[{'id': '1', 'model': 'AR', 'num_r': 4, 'rate': 1.0, 'dis': 100, 'action_bound': 2, 'data_buf_size': 100, 't_factor': 0.5, 'penalty': 1000}, {'id': '2', 'model': 'AR', 'num_r': 4, 'rate': 2.0, 'dis': 100, 'action_bound': 2, 'data_buf_size': 100, 't_factor': 0.5, 'penalty': 1000}, {'id': '3', 'model': 'AR', 'num_r': 4, 'rate': 3.0, 'dis': 100, 'action_bound': 2, 'data_buf_size': 100, 't_factor': 0.5, 'penalty': 1000}]
INFO:tensorflow:Restoring parameters from model_M_05_nB/my_train_model_0-2000
Initialization OK!----> user 1
INFO:tensorflow:Restoring parameters from model_M_05_nB/my_train_model_0-2000
Initialization OK!----> user 2
INFO:tensorflow:Restoring parameters from model_M_05_nB/my_train_model_0-2000
Initialization OK!----> user 3
0:r:[ -1.93227103  -5.95293982 -12.94022502],p:[0.22232339 0.69250808 1.95515021],op:[4.20228255e-03 1.82950967e-03 3.00082868e-18],tr:[0.16171585 0.63493303 1.50055514],pr:[0.905724   1.35650288 1.51638702],rev:[0.9884 1.9866 3.

21:r:[ -1.95565253  -5.88149025 -12.76768362],p:[0.22499023 0.68605643 1.93494687],op:[0.00435823 0.00189699 0.        ],tr:[0.15774023 0.66572359 1.46217053],pr:[0.90960298 1.34522148 1.51869696],rev:[0.9891 2.0051 2.9807],dbuf:[1.61782046 4.88344629 6.18589853],ch:[3.98277794 4.07597606 3.93909635],ibuf:[0, 0, 0],rbuf:[5.0754324  6.20966102 2.28210337]
22:r:[ -1.91292961  -5.96863203 -13.06618364],p:[0.22027278 0.69570079 1.97669457],op:[0.00417295 0.00223852 0.        ],tr:[0.16309592 0.6670729  1.48953598],pr:[0.90384534 1.35180065 1.54293884],rev:[0.9881 2.0115 3.0327],dbuf:[1.58140188 4.95787096 6.36542155],ch:[3.93440309 3.96871497 3.95989888],ibuf:[0, 0, 0],rbuf:[1.         3.34723155 4.53393211]
23:r:[ -1.93249458  -5.87099392 -12.82045509],p:[0.22222376 0.68512646 1.94735044],op:[0.00423167 0.00185595 0.        ],tr:[0.16071452 0.65421844 1.4797477 ],pr:[0.90939801 1.35429173 1.52462948],rev:[0.995  2.0022 3.0048],dbuf:[1.60043487 4.87216375 6.16740575],ch:[3.98572295 4.09736

44:r:[ -1.99037881  -5.89292864 -12.71478043],p:[0.22980821 0.68654557 1.9309345 ],op:[0.00423304 0.0018277  0.        ],tr:[0.1631065  0.67556624 1.50498498],pr:[0.92036908 1.34335449 1.49287762],rev:[1.0066 2.0126 2.9977],dbuf:[1.64034514 4.90212461 6.12021591],ch:[3.93047158 4.14074819 4.02441457],ibuf:[0, 0, 0],rbuf:[0.72373382 3.08879102 8.81242981]
45:r:[ -1.94008573  -5.74940455 -12.95068339],p:[0.2232362  0.67594953 1.96831538],op:[0.00423179 0.00228927 0.        ],tr:[0.1622959  0.64582552 1.46316433],pr:[0.91094162 1.34269755 1.55600079],rev:[0.9966 1.9818 3.0195],dbuf:[1.60549157 4.71642119 6.21821295],ch:[4.02288156 3.90859189 3.82793391],ibuf:[0, 0, 0],rbuf:[ 0.          1.71449462 12.16122078]
46:r:[ -1.99981082  -5.82526837 -12.66655687],p:[0.23105643 0.67903589 1.92030479],op:[0.00432323 0.00215088 0.        ],tr:[0.16556721 0.66005382 1.50880234],pr:[0.91834532 1.34266631 1.49279573],rev:[1.0066 1.9959 3.0013],dbuf:[1.64582509 4.83866904 6.13006587],ch:[4.02142997 4.03

67:r:[ -1.96920015  -6.0075277  -12.74764757],p:[0.2265336  0.69772568 1.92790538],op:[0.00404259 0.00169201 0.        ],tr:[0.16574689 0.65088839 1.51158876],pr:[0.90866476 1.3580271  1.4855431 ],rev:[0.9981 2.0042 2.9977],dbuf:[1.63263843 5.02087853 6.21624134],ch:[3.76477773 3.99848426 4.14491149],ibuf:[0, 0, 0],rbuf:[ 1.31772048 12.09917439  8.59606533]
68:r:[ -2.00431569  -5.86151641 -12.90833918],p:[0.23147331 0.68466908 1.95667792],op:[0.00395519 0.00197112 0.        ],tr:[0.16487692 0.65585708 1.51089987],pr:[0.92362455 1.34839345 1.51674792],rev:[1.0162 1.9974 3.0273],dbuf:[1.65434639 4.85663083 6.24989913],ch:[4.16546522 4.05814973 4.02648093],ibuf:[0, 0, 0],rbuf:[2.         1.27004964 5.11820898]
69:r:[ -2.02191797  -5.95141328 -12.63570322],p:[0.23448274 0.69441973 1.92430238],op:[0.00397884 0.00179812 0.        ],tr:[0.17401471 0.66364727 1.51057758],pr:[0.90797567 1.35355762 1.49509155],rev:[1.0066 2.0112 3.0062],dbuf:[1.65922012 4.94064814 6.02838265],ch:[3.98389485 4.04

90:r:[ -1.94621198  -6.1841132  -12.8035681 ],p:[0.22414946 0.72072778 1.94092891],op:[0.00406327 0.00182574 0.        ],tr:[0.16271457 0.66713367 1.47501904],pr:[0.90918579 1.37556549 1.51662576],rev:[0.9964 2.037  2.9934],dbuf:[1.61029666 5.14269128 6.19784707],ch:[4.1322417  3.96256649 3.96543832],ibuf:[0, 0, 0],rbuf:[ 0.15027784  2.06515674 19.60046101]
91:r:[ -2.0198981   -6.01283247 -12.85279881],p:[0.23331633 0.69796173 1.94625537],op:[0.00414912 0.00196563 0.        ],tr:[0.17141857 0.6546765  1.50121744],pr:[0.91751903 1.35752561 1.50425265],rev:[1.0136 2.0058 3.0046],dbuf:[1.66514167 5.02639133 6.24304393],ch:[3.88288919 4.14241375 4.08549044],ibuf:[0, 0, 0],rbuf:[ 1.64085424  2.4699145  10.89955011]
92:r:[ -1.91324776  -5.88904452 -12.74845962],p:[0.21953337 0.68953331 1.92951697],op:[0.00421    0.00198286 0.        ],tr:[0.15862122 0.63853327 1.47870035],pr:[0.90541255 1.35283934 1.49769277],rev:[0.987  1.9853 2.9766],dbuf:[1.58906184 4.8629273  6.20174953],ch:[3.96880987 3

...

97:r:[ -1.97508569  -5.5507049  -13.71118757],p:[0.19940546 0.75502535 1.99055139],op:[0.00406415 0.0008757  0.        ],tr:[0.26153886 0.78786987 1.43384523],pr:[0.81644103 1.20653062 1.58880491],rev:[1.0031 1.9849 3.0217],dbuf:[1.91547533 3.54239935 7.51686127],ch:[3.94031596 4.24811488 4.04917713],ibuf:[0, 0, 0],rbuf:[3.07153223 3.78855681 5.77425224]
98:r:[ -1.87427699  -5.73532566 -13.45935901],p:[0.18999524 0.78099151 1.9560736 ],op:[0.00431216 0.00082218 0.        ],tr:[0.26198328 0.77807969 1.46083729],pr:[0.80981037 1.2425881  1.55503923],rev:[0.9933 2.0119 3.0158],dbuf:[1.80547997 3.65251438 7.35798204],ch:[4.13252877 3.97242776 4.03171053],ibuf:[0, 0, 0],rbuf:[3.29991386 2.48495498 5.00903694]
99:r:[ -1.92265007  -5.75253435 -13.7006997 ],p:[0.19356624 0.78278198 1.99144271],op:[0.00426306 0.00081326 0.        ],tr:[0.25561538 0.78284197 1.42014261],pr:[0.81048401 1.23725034 1.58849457],rev:[0.9856 2.0115 3.009 ],dbuf:[1.86700719 3.66911625 7.48697228],ch:[4.04494183 4.18093

18:r:[ -1.89691858  -5.60838289 -12.42888309],p:[0.17566865 0.7041415  1.59524238],op:[0.00152325 0.00308403 0.00665338],tr:[0.11161613 0.63124547 1.31889325],pr:[0.91652937 1.37989184 1.69042878],rev:[1.0113 1.993  2.9922],dbuf:[2.02191815 4.14451049 8.83880855],ch:[4.0170044  4.03534052 4.02144986],ibuf:[0, 0, 0],rbuf:[1.50734455 6.94915501 7.84649718]
19:r:[ -1.90224318  -5.72921256 -12.56830682],p:[0.17657699 0.71780958 1.60589275],op:[0.00162581 0.00315596 0.00553917],tr:[0.10962916 0.63345598 1.32748232],pr:[0.92445843 1.38617135 1.70232997],rev:[1.0171 2.0003 3.0158],dbuf:[2.02245834 4.24876969 9.02229443],ch:[3.96805802 3.96286817 4.05669234],ibuf:[0, 0, 0],rbuf:[1.26494236 3.4359183  8.55050602]
20:r:[ -1.87289541  -5.63408837 -12.85757785],p:[0.17334573 0.70566355 1.63102763],op:[0.00167317 0.00303989 0.00600544],tr:[0.10890403 0.61677486 1.30511526],pr:[0.91169537 1.3859733  1.71620284],rev:[1.0031 1.9849 3.0058],dbuf:[1.99560184 4.1811423  9.34482501],ch:[4.00378046 3.96810

41:r:[ -1.85505373  -5.81295985 -12.4577277 ],p:[0.17173374 0.72307347 1.5974115 ],op:[0.0017954  0.00298397 0.00608928],tr:[0.10838152 0.63025915 1.28762969],pr:[0.91333101 1.39803904 1.70923841],rev:[1.0016 2.0107 2.9811],dbuf:[1.97481606 4.36534533 8.88044758],ch:[4.09005133 3.87951801 3.84698199],ibuf:[0, 0, 0],rbuf:[2.33509499 3.85808619 3.30081804]
42:r:[ -1.85177808  -5.57119901 -12.56811296],p:[0.17170655 0.69963589 1.60684809],op:[0.00166584 0.00322794 0.00660488],tr:[0.10610674 0.6366354  1.31207198],pr:[0.91157624 1.37555151 1.69397336],rev:[0.9997 1.9935 2.9898],dbuf:[1.96983234 4.11375975 9.00169621],ch:[4.00845509 4.06252682 4.00258541],ibuf:[0, 0, 0],rbuf:[1.03408568 3.52127475 8.01452654]
43:r:[ -1.84450548  -5.66724806 -12.50482755],p:[0.17092662 0.71158131 1.59771904],op:[0.00179672 0.00318298 0.00623158],tr:[0.09547202 0.62495613 1.31424651],pr:[0.91566818 1.38573305 1.6913463 ],rev:[0.9921 1.9914 2.9894],dbuf:[1.96177755 4.18685322 8.97014884],ch:[3.84478314 3.91498

64:r:[ -1.82005503  -5.57665661 -12.4928133 ],p:[0.1686238  0.69983542 1.60097597],op:[0.00169372 0.00344021 0.00595085],tr:[0.10086295 0.63518571 1.3188011 ],pr:[0.90688453 1.38148287 1.69750339],rev:[0.9893 1.9968 2.9999],dbuf:[1.93693478 4.12055698 8.91635841],ch:[3.96670068 3.93695603 3.99898598],ibuf:[0, 0, 0],rbuf:[0.13249657 1.56220351 5.19600235]
65:r:[ -1.88097936  -5.76737315 -12.72450705],p:[0.174      0.72064395 1.6198675 ],op:[0.00199766 0.00295297 0.00610795],tr:[0.11102848 0.62249332 1.3252398 ],pr:[0.91295734 1.39403747 1.69350268],rev:[1.0044 1.9994 3.0037],dbuf:[2.00198211 4.29877707 9.18925955],ch:[3.99761417 3.87961721 4.15144482],ibuf:[0, 0, 0],rbuf:[4.41360061 4.97845219 7.39041294]
66:r:[ -1.85643187  -5.62651064 -12.87425668],p:[0.17216337 0.70310871 1.63385643],op:[0.00183108 0.00296196 0.00639689],tr:[0.1029087  0.59822807 1.27985528],pr:[0.91392658 1.39038257 1.72411765],rev:[0.997  1.9703 2.9894],dbuf:[1.97291923 4.19231455 9.34598018],ch:[3.90110231 3.86498

87:r:[ -1.84942518  -5.81404664 -13.06687867],p:[0.17126174 0.72806929 1.65697265],op:[0.00174641 0.00254619 0.00496062],tr:[0.10110495 0.65087098 1.31520775],pr:[0.90925899 1.39179552 1.73153411],rev:[0.9911 2.0268 3.0337],dbuf:[1.96876887 4.32193854 9.51442469],ch:[3.90955905 4.0814465  3.98043775],ibuf:[0, 0, 0],rbuf:[2.34862669 2.24072431 2.        ]
88:r:[ -1.90833806  -5.72512561 -12.84148313],p:[0.17659246 0.7180441  1.63337809],op:[0.00169684 0.0029592  0.00564456],tr:[0.11037256 0.64620789 1.29723257],pr:[0.92364234 1.38361317 1.72713847],rev:[1.0153 2.0119 3.01  ],dbuf:[2.03378321 4.24021824 9.29273977],ch:[4.02350202 4.05450382 3.94534053],ibuf:[0, 0, 0],rbuf:[1.85258123 4.         5.85715067]
89:r:[ -1.83202939  -5.54226737 -12.30826933],p:[0.16892504 0.69673486 1.58060051],op:[0.00134759 0.0034545  0.00594727],tr:[0.11035197 0.63792674 1.29009099],pr:[0.9035755  1.37145484 1.69318342],rev:[0.9989 1.9889 2.9686],dbuf:[1.96133256 4.08264119 8.75106094],ch:[4.31607158 4.12638

In [2]:
name = res_path+'/test_1000_' + time.strftime("%b_%d_%Y_%H_%M_%S", time.localtime(time.time()))
# print(name)
np.savez(name, res_r, res_p, res_b, res_o, res_d)

In [ ]:
graph = tf.get_default_graph()
# variable_names = [v.name for v in tf.trainable_variables()]
# values = sess.run(variable_names)
# tf.trainable_variables()

inputs = graph.get_tensor_by_name("input_1/X"+":0")
scaled_out = graph.get_tensor_by_name("output_1"+":0")
print sess.run(scaled_out, feed_dict={inputs: [[7.22254768,3.9729721,0.7729721]]})
#Create a saver object which will save all the variables
saver = tf.train.Saver()
 
#Now, save the graph
saver.save(sess, 'model/my_train_model',global_step=2000)

# user_list[0].agent.actor.predict([[7.22254768,3.9729721,0.7729721]]) # [[0.63916105, 0.20230901]]